In [1]:
from numpy import reshape
import numpy as np
import seaborn as sns
import pandas as pd  
import torch
import os

from tqdm import tqdm
from pathlib import Path
import json
from unittest import result
import torch
import numpy as np
from scipy.io.wavfile import write
from env import AttrDict
from meldataset import mel_spectrogram, MAX_WAV_VALUE, load_wav
from hifigan import Generator, MultiPeriodDiscriminator, MultiScaleDiscriminator
from utils import load_checkpoint
import random
import soundfile as sf
MAX_WAV_VALUE = 32768.0
import argparse
from pathlib import Path
from concurrent.futures import ProcessPoolExecutor
from multiprocessing import cpu_count
from tqdm import tqdm

In [2]:
# Feature extract
def discriminator_loss(disc_real_outputs, disc_generated_outputs):
    loss = 0
    r_losses = []
    g_losses = []
    for dr, dg in zip(disc_real_outputs, disc_generated_outputs):
        r_loss = torch.mean((1-dr)**2)
        g_loss = torch.mean(dg**2)
        loss += (r_loss + g_loss)
        r_losses.append(r_loss.item())
        g_losses.append(g_loss.item())

    return loss, r_losses, g_losses

def feature_loss(fmap_r, fmap_g):
    loss = 0
    for dr, dg in zip(fmap_r, fmap_g):
        for rl, gl in zip(dr, dg):
            loss += torch.mean(torch.abs(rl - gl))

    return loss*2

class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

def get_mel(x,sr,h):
    return mel_spectrogram(y=x, n_fft=h.n_fft, num_mels=h.num_mels, sampling_rate = sr, hop_size=h.hop_size, win_size=h.win_size, fmin=h.fmin, fmax=sr/2.0)

def load_flac(full_path):
    data, sampling_rate = sf.read(full_path)
    return data, sampling_rate


In [3]:
config_file = "/dataa/phucdt/vocodetect/hifi-gan/cp_16k/config.json"
do_file = "/dataa/phucdt/vocodetect/hifi-gan/cp_16k/do_00255000"
g_file = "/dataa/phucdt/vocodetect/hifi-gan/cp_16k/g_00255000"
device = "cuda:0"

with open(config_file) as f:
    data = f.read()
json_config = json.loads(data)
h = AttrDict(json_config)

generator = Generator(h).to(device)

state_dict_g = load_checkpoint(g_file, device)
generator.load_state_dict(state_dict_g['generator'])

mpd = MultiPeriodDiscriminator().to(device)
msd = MultiScaleDiscriminator().to(device)

state_dict_do = load_checkpoint(do_file, device)

mpd.load_state_dict(state_dict_do['mpd'])
msd.load_state_dict(state_dict_do['msd'])

mpd.eval()
msd.eval()

Loading '/dataa/phucdt/vocodetect/hifi-gan/cp_16k/g_00255000'
Complete.
Loading '/dataa/phucdt/vocodetect/hifi-gan/cp_16k/do_00255000'
Complete.


MultiScaleDiscriminator(
  (discriminators): ModuleList(
    (0): DiscriminatorS(
      (convs): ModuleList(
        (0): Conv1d(1, 128, kernel_size=(15,), stride=(1,), padding=(7,))
        (1): Conv1d(128, 128, kernel_size=(41,), stride=(2,), padding=(20,), groups=4)
        (2): Conv1d(128, 256, kernel_size=(41,), stride=(2,), padding=(20,), groups=16)
        (3): Conv1d(256, 512, kernel_size=(41,), stride=(4,), padding=(20,), groups=16)
        (4): Conv1d(512, 1024, kernel_size=(41,), stride=(4,), padding=(20,), groups=16)
        (5): Conv1d(1024, 1024, kernel_size=(41,), stride=(1,), padding=(20,), groups=16)
        (6): Conv1d(1024, 1024, kernel_size=(5,), stride=(1,), padding=(2,))
      )
      (conv_post): Conv1d(1024, 1, kernel_size=(3,), stride=(1,), padding=(1,))
    )
    (1): DiscriminatorS(
      (convs): ModuleList(
        (0): Conv1d(1, 128, kernel_size=(15,), stride=(1,), padding=(7,))
        (1): Conv1d(128, 128, kernel_size=(41,), stride=(2,), padding=(20,), g

In [4]:
def pad(x, max_len=64600):
    x_len = x.shape[0]
    if x_len >= max_len:
        return x[:max_len]
    # need to pad
    num_repeats = int(max_len / x_len)+1
    padded_x = np.tile(x, (1, num_repeats))[:, :max_len][0]
    return padded_x	

In [63]:
audio_path = "/dataa/Dataset/ASVspoof/LA/ASVspoof2019_LA_train/flac/LA_T_1272637.flac"
wav, sr = load_flac(audio_path)
wav = pad(wav,max_len=64600)
wav = torch.FloatTensor(wav).to(device)
wav = wav.unsqueeze(0).unsqueeze(1)

y_d_rs, y_d_gs, fmap_d_rs, fmap_d_gs = mpd(wav,wav.detach())
y_s_rs, y_s_gs, fmap_s_rs, fmap_s_gs = msd(wav,wav.detach())
emb_d = torch.flatten(fmap_d_rs[-1][-1], 1, -1).detach().squeeze()
emb_s = torch.flatten(fmap_s_rs[-1][-1], 1, -1).detach().squeeze()

In [65]:
fmap_s_rs[0][6].transpose(1, 2).shape

torch.Size([1, 1010, 1024])

In [17]:
x = torch.cat((emb_d, emb_s),0)
x.shape

torch.Size([143])

In [7]:
tmp = torch.Tensor().to(device)
max_len = len(y_d_rs[0][0])
print(max_len)
for i in y_d_rs:
    print(i[0,:max_len].shape)
    tmp=torch.cat((tmp,i[0,:max_len]),dim=-1)
    
# y_d_rs[0][0].detach().cpu().shape
tmp=tmp.reshape((5,102))
tmp.shape


102
torch.Size([102])
torch.Size([102])
torch.Size([102])
torch.Size([102])
torch.Size([102])


torch.Size([5, 102])

In [8]:
x = torch.load("./feats/2019_mpd_ys_102/LA_D_1000265.flac")
x.shape

torch.Size([5, 102])

In [8]:
mypath = Path("/dataa/phucdt/vocodetect/feats/2019_mpd_cat_msd_y_143")
feat = pd.DataFrame()
# feat.columns = ["utt", "feats"]
emb_paths = mypath.rglob("*.flac")
for f in tqdm(emb_paths):
    # print(f)
    relative_path = f.relative_to(mypath)
    utt = os.path.basename(f).replace(".flac","")
    emb = torch.load(f).cpu().detach().numpy()
    # feat = feat.append({"utt": utt, "feats": emb}, ignore_index=True)
    # # emb = torch.view_as_real(emb
    newrow = pd.Series({"utt": utt, "feats": emb})
    feat = pd.concat([feat, newrow.to_frame().T], ignore_index=True)
    # print(bio_inp)


feat.head()

50366it [00:53, 935.06it/s] 


,utt,feats
0,LA_T_7357347,"[0.6257009, 0.59887695, 0.5864012, 0.58410853,..."
1,LA_D_5081966,"[0.47547257, 0.464983, 0.45322007, 0.45323932,..."
2,LA_T_5310424,"[0.8715479, 0.9270203, 0.8975519, 0.88319826, ..."
3,LA_T_2607453,"[0.922325, 0.9768211, 0.9993576, 0.9219039, 0...."
4,LA_T_5194064,"[0.6673494, 0.6926455, 0.7289912, 0.72983927, ..."


In [32]:
mypath = Path("/dataa/Dataset/VCTK/")
emb_paths = mypath.rglob("*.wav")
list_file = []
for f in tqdm(emb_paths):
    relative_path = f.relative_to(mypath)
    list_file.append(str(relative_path).replace(".wav","")+"|aaaa\n")

44242it [00:00, 48888.29it/s]


In [33]:
str(list_file[0])

'p256/p256_268|aaaa\n'

In [34]:
import random
random.shuffle(list_file)
train_list = list_file[:40000]
eval_list = list_file[40000:]


In [35]:
with open("./hifi-gan/VCTK/train.txt", "w") as f:
    f.writelines(train_list)
with open("./hifi-gan/VCTK/test.txt", "w") as f:
    f.writelines(eval_list)

In [9]:
# Training set
df = pd.read_csv('/dataa/Dataset/ASVspoof/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.train.trn.txt', sep=' ')
df.columns = ["sid", "utt", "phy", "attack", "class"]
train = df.join(feat.set_index('utt'), on="utt", how = "inner")
train.head()

,sid,utt,phy,attack,class,feats
0,LA_0079,LA_T_1271820,-,-,bonafide,"[0.491719, 0.44580877, 0.41168386, 0.4468835, ..."
1,LA_0079,LA_T_1272637,-,-,bonafide,"[0.6733224, 0.7313524, 0.7296496, 0.8248597, 0..."
2,LA_0079,LA_T_1276960,-,-,bonafide,"[0.44089127, 0.44330055, 0.4449181, 0.44075567..."
3,LA_0079,LA_T_1341447,-,-,bonafide,"[0.46057808, 0.44826508, 0.41439158, 0.4344164..."
4,LA_0079,LA_T_1363611,-,-,bonafide,"[0.43592113, 0.41540164, 0.44281036, 0.4542095..."


In [10]:
# Dev set
df = pd.read_csv('/dataa/Dataset/ASVspoof/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.dev.trl.txt', sep=' ')
df.columns = ["sid", "utt", "phy", "attack", "class"]
dev = df.join(feat.set_index('utt'), on="utt", how = "inner")
dev.head()

,sid,utt,phy,attack,class,feats
0,LA_0069,LA_D_1105538,-,-,bonafide,"[0.49437612, 0.48988664, 0.4504388, 0.4695245,..."
1,LA_0069,LA_D_1125976,-,-,bonafide,"[0.5270141, 0.5310479, 0.53959227, 0.5427851, ..."
2,LA_0069,LA_D_1293230,-,-,bonafide,"[0.5992119, 0.6958335, 0.61461896, 0.67813784,..."
3,LA_0069,LA_D_1340209,-,-,bonafide,"[0.8440463, 0.8074619, 0.7218627, 0.72035813, ..."
4,LA_0069,LA_D_1376638,-,-,bonafide,"[0.5676229, 0.537273, 0.5739882, 0.5865316, 0...."


In [11]:
# Eval set
df = pd.read_csv('/dataa/Dataset/ASVspoof/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.eval.trl.txt', sep=' ')
df.columns = ["sid", "utt", "phy", "attack", "class"]
eval = df.join(feat.set_index('utt'), on="utt", how = "inner")
eval.head()

,sid,utt,phy,attack,class,feats


# Random Forest

In [12]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=16)

In [13]:
dev_train = train.append(dev)
X = train['feats'].to_list()
Y = train['class'].to_numpy()
clf.fit(X,Y)

/tmp/ipykernel_1195674/2263313023.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dev_train = train.append(dev)


RandomForestClassifier(n_jobs=16, random_state=0)

In [15]:
X_test = dev['feats'].to_list()
Y_test = dev['class'].to_numpy()
clf.score(X_test, Y_test)

0.8975969085859196

# SVM

In [80]:
from sklearn import svm
scl = svm.SVC(probability=True)
scl.fit(X,Y)

SVC(probability=True)

In [83]:
scl.score(X_test, Y_test)

0.8967516424279859

In [79]:
from sklearn import decomposition
import matplotlib.pyplot as plt
print("calculating pca")
pca = decomposition.PCA(n_components=2)
pca.fit(X)
z = pca.transform(X)

calculating pca


In [85]:
import matplotlib.pyplot as plt

In [90]:
dff = pd.DataFrame()
dff["y"] = Y
dff["comp-1"] = z[:,0]
dff["comp-2"] = z[:,1]

sns.scatterplot(x="comp-1", y="comp-2", hue=dff.y.tolist(),
                palette=sns.color_palette("hls", 2),
                data=dff).set(title="") 

plt.show()
# plt.savefig('test.png',dpi=600)

In [36]:
from model import HifiDis

In [37]:
audio_path = "/dataa/Dataset/ASVspoof/LA/ASVspoof2019_LA_train/flac/LA_T_1272637.flac"
wav, sr = load_flac(audio_path)

In [38]:
model = HifiDis(device="cuda:0")

In [39]:
feat = model.extract_feat(wav)

AttributeError: 'numpy.ndarray' object has no attribute 'device'